```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import Drifter, CPUDrifter, GPUDrifter
from SWESimulators import Resampling

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

## Running multiple Simulators with different wind direction

While keeping a track on the drifters...

In this case, each particle is a combination of a unique drifter living in a unique model state.

Ideas that might evolve into code:
- It should be optional for a Drifter to have an observation. An additional `.hasObservation()` method needs to be implemented, and included somewhere.
- Does it make sense to have a set of Drifters without an Ocean model? They could live in a steady state environment where the u-v-fields are given from a netCDF? Or simply just from a numpy.array. 

In [ ]:
reload(GPUDrifter)
reload(CDKLM16)
reload(Common)

class OceanStateWithDrifters:
    def __init__(self, sim, drifters):
        self.sim = sim
        assert isinstance(drifters, Drifter), 'drifters is type: ' + type(drifters) + \
                ', but has to be a Drifter-type instance.'
            
        self.drifters = drifters
    
    

class OceanStateEnsamble:
    def __init__(self, numParticles, cl_ctx):
        
        self.cl_ctx = cl_ctx
        
        self.numParticles = numParticles
        self.particles = [None]*(self.numParticles + 1)
        
        self.obs_index = self.numParticles
        
        self.simType = 'CDKLM16'
        
    def cleanUp(self):
        for oceanState in self.particles:
            if oceanState is not None:
                oceanState.cleanUp()
        
    def setGridInfo(self, nx, ny, dx, dy, dt, boundaryConditions):
        self.nx = nx
        self.ny = ny
        self.dx = dx
        self.dy = dy
        self.dt = dt
        
        self.observation_variance = 3*dx
        
        self.initialization_variance = 10*dx
        self.midPoint = 0.5*np.array([self.nx*self.dx, self.ny*self.dy])
        self.initialization_cov = np.eye(2)*self.initialization_variance
        
        self.boundaryConditions = boundaryConditions
        
        assert(self.simType == 'CDKLM16'), 'CDKLM16 is currently the only supported scheme'
        #if self.simType == 'CDKLM16':
        self.ghostCells = np.array([2,2,2,2])
        if self.boundaryConditions.isSponge():
            sponge = self.boundaryConditions.getSponge()
            for i in range(4):
                if sponge[i] > 0: 
                    self.ghostCells[i] = sponge[i]
        dataShape =  ( ny + self.ghostCells[0] + self.ghostCells[2], 
                       nx + self.ghostCells[1] + self.ghostCells[3]  )
            
        # Create base initial data:
        self.base_eta = np.zeros(dataShape, dtype=np.float32, order='C')
        self.base_hu  = np.zeros(dataShape, dtype=np.float32, order='C');
        self.base_hv  = np.zeros(dataShape, dtype=np.float32, order='C');
        
        # Bathymetry:
        waterDepth = 5
        self.base_Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*waterDepth
 
    
    def setParameters(self, f, g=9.81, beta=0, r=0, wind=Common.WindStressParams(type=99)):
        self.g = g
        self.f = f
        self.beta = beta
        self.r = r
        self.wind = wind
    
    def initWindCase(self, driftersPerOceanModel=1):
        self.windSpeed = 5.0
        self.directions = np.random.rand(self.numParticles + 1)*360*0
        print "Directions: ", self.directions
        
        for i in range(self.numParticles+1):
            wind = Common.WindStressParams(type=50, 
                                           wind_speed=self.windSpeed,
                                           wind_direction=self.directions[i])

            
            self.particles[i] = CDKLM16.CDKLM16(self.cl_ctx, \
                                                self.base_eta, self.base_hu, self.base_hv, \
                                                self.base_Hi, \
                                                self.nx, self.ny, self.dx, self.dy, self.dt, \
                                                self.g, self.f, self.r, \
                                                wind_stress=wind, \
                                                boundary_conditions=self.boundaryConditions, \
                                                write_netcdf=False)
            
            drifters = GPUDrifter.GPUDrifter(cl_ctx, driftersPerOceanModel,
                                             observation_variance=self.observation_variance,
                                             boundaryConditions=self.boundaryConditions,
                                             domain_size_x=self.nx*self.dx, domain_size_y=self.ny*self.dy)
            initPos = np.random.multivariate_normal(self.midPoint, self.initialization_cov, driftersPerOceanModel)
            drifters.setParticlePositions(initPos)
            #print "drifter particles: ", drifter.getParticlePositions()
            #print "drifter observations: ", drifter.getObservationPosition()
            self.particles[i].attachDrifters(drifters)
        
    
    def getParticlePositions(self):
        drifterPositions = np.empty((0,2))
        for oceanState in self.particles:
            drifterPositions = np.append(drifterPositions, 
                                         oceanState.drifters.getParticlePositions(),
                                         axis=0)
        return drifterPositions
    
    def step(self, t):
        simNo = 0
        for oceanState in self.particles:
            #print "Starting sim " + str(simNo)
            oceanState.step(t)
            #print "Finished sim " + str(simNo)      
            simNo = simNo + 1
            
    def printMaxOceanStates(self):
        simNo = 0
        for oceanState in self.particles:
            eta, hu, hv = oceanState.download()
            print "------- simNo: " + str(simNo) + " -------"
            print "Max eta: ", np.max(eta)
            print "Max hu:  ", np.max(hu)
            print "Max hv:  ", np.max(hv)
            

In [ ]:
nx, ny, dx, dy = 50, 50, 4.0, 4.0
dt = 1
f = 0.1
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

if 'ensemble' in globals():
    ensemble.cleanUp()

numParticles = 5
ensemble = OceanStateEnsamble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

particlesPerSim = 5
ensemble.initWindCase(particlesPerSim)
startPos = ensemble.getParticlePositions()
#print "StartPos:\n", startPos
fig = plt.figure(figsize=(5,5))
plt.plot(startPos[:,0], startPos[:,1], 'yo')
plt.ylim([0, ny*dy])
plt.xlim([0, nx*dx])

for i in range(10):
    print "iteration " + str(i)
    ensemble.step(500)
    pos = ensemble.getParticlePositions()
    plt.plot(pos[:,0], pos[:,1], 'r.')
    #print pos

In [ ]:
ensemble.printMaxOceanStates()